<a href="https://colab.research.google.com/github/FunmilolaAdetuwo/Final-Project/blob/main/Analyzing_the_Impact_of_Mergers_and_Acquisitions_on_Stock_Prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Importing necessary libraries

import yfinance as yf
import pandas as pd
from datetime import datetime

In [3]:
# Let's list the name of the companies for the different Merger and Acquisition(M&A) types

# For the stock price years we will be using 5 year stock price, which will be 2.5 years pre M&A date and 2.5 years post M&A dates

companies = {
    'horizontal': {'company': 'MSFT', 'ma_dte': '2016-06-13', 'start_date': '2013-12-13', 'end_date': '2018-12-13'},  # Microsoft acquiring LinkedIn
    'vertical': {'company': 'AMZN', 'ma_dte': '2017-06-16', 'start_date': '2014-12-16', 'end_date': '2019-12-16'},    # Amazon acquiring Whole Foods
    'conglomerate': {'company': 'BRK.A', 'ma_dte': '2015-08-10', 'start_date': '2013-02-10', 'end_date': '2018-02-10'}  # Berkshire Hathaway acquiring Precision Castparts
}



In [ ]:
# Download stock price data from Yahoo Finance for 2.5 years before and after
def download_stock_data(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    return stock_data

# Collect stock data for each M&A type
stock_data = {}
for mna_type, company_info in companies.items():
    ticker = company_info['company']
    start_date = company_info['start_date']
    end_date = company_info['end_date']
    stock_data[mna_type] = download_stock_data(ticker, start_date, end_date)

# Combine with M&A dates
mna_dates = pd.DataFrame(companies).T
mna_dates['mna_date'] = pd.to_datetime(mna_dates['mna_date'])

# Display the stock data for validation
print(stock_data)